In [19]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np

In [20]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

def set_col_types(df):
    if "target" in df.columns:
        categorical_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68', 'target']
    else:
        categorical_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
    df['customer_ID'] = df['customer_ID'].astype("string")
    
    for col in categorical_cols:
        df[col] = df[col].astype("string")
    df["S_2"] = pd.to_datetime(df['S_2'], format=r'%Y-%m-%d').astype('datetime64[ns]')
    df["B_31"] = df["B_31"].astype(np.int8)
    return df

def sync_cols(train_df, pred_df):
    for col in train_df.columns:
      if col not in pred_df.columns:
        print(col, "not in pred_df so adding - should always be categorical!")
        pred_df[col] = 0
    for col in pred_df.columns:
      if col not in train_df.columns:
        print(col, "not in train_df so dropping")
        pred_df = pred_df.drop(col, axis=1)
    return pred_df

In [21]:
# Read data from parquet file
df = pd.read_parquet(r"../../../amex-default-prediction/train_data.parquet")
#reduce df for development !!!!! comment out line below for final model
#df = df[:100000]

# Set the data types for the columns
df = set_col_types(df)

#engineer statement num
df['statement_num'] = df.groupby("customer_ID")['S_2'].rank(method='first', ascending=False).astype(np.int8) 
df = df[df["statement_num"] == 1]
df.reset_index(inplace=True, drop=True)
#engineer date cols
df["Month"] = df["S_2"].dt.month
df["Day"] = df["S_2"].dt.day
df["Year"] = df["S_2"].dt.year
df = df.drop(["S_2"], axis=1)

# Separate target variable and feature columns
target = df["target"].astype(int)
labels = df['customer_ID']
features = df.drop(["customer_ID", "target"], axis=1)

# Impute missing values using mode for categorical columns and median for numerical columns
cat_columns = features.select_dtypes(include=["string"]).columns
num_columns = features.select_dtypes(include="number").columns

# Replace missing values in the categorical columns with the most frequent value
for col in cat_columns:
    features[col].fillna("NA", inplace=True)

# Replace missing values in the numerical columns with the median value
for col in num_columns:
    features[col].fillna(features[col].mean(), inplace=True)

features = pd.get_dummies(features)

features = features.sort_index(axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

# Create XGBoost model
model = xgb.XGBClassifier()
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the performance of the model
print(classification_report(y_test, y_pred, digits=5))

              precision    recall  f1-score   support

           0    0.92961   0.93295   0.93128    102185
           1    0.80493   0.79658   0.80074     35489

    accuracy                        0.89780    137674
   macro avg    0.86727   0.86477   0.86601    137674
weighted avg    0.89747   0.89780   0.89763    137674



In [22]:
amex_test = y_test.reset_index(drop=True).to_frame()
amex_test['target'] = amex_test['target'].astype('int')

amex_pred = model.predict_proba(X_test)
amex_pred = pd.DataFrame(amex_pred,columns=["proba-inv","proba"])
amex_pred = amex_pred.drop(columns=['proba-inv'])
amex_pred = amex_pred.rename(columns={"proba":"prediction"})


amex_metric(amex_test, amex_pred)

0.7742451693185308

In [23]:
# Read test data from parquet file
df = pd.read_parquet(r"../../../amex-default-prediction/test_data.parquet")
#df = df[:100000]

# Set the data types for the columns
df = set_col_types(df)

# Engineer statement num
df['statement_num'] = df.groupby("customer_ID")['S_2'].rank(method='first', ascending=False).astype(np.int8)
df = df[df["statement_num"] == 1]
df.reset_index(inplace=True, drop=True)
# Engineer date cols
df["Month"] = df["S_2"].dt.month
df["Day"] = df["S_2"].dt.day
df["Year"] = df["S_2"].dt.year
df = df.drop(["S_2"], axis=1)

# Separate labels and feature columns
labels = df['customer_ID']
features = df.drop(["customer_ID"], axis=1)

# Impute missing values using mode for categorical columns and median for numerical columns
cat_columns = features.select_dtypes(include=["string"]).columns
num_columns = features.select_dtypes(include="number").columns

# Replace missing values in the categorical columns with the most frequent value
for col in cat_columns:
    features[col].fillna("NA", inplace=True)

# Replace missing values in the numerical columns with the median value
for col in num_columns:
    features[col].fillna(features[col].mean(), inplace=True)

features = pd.get_dummies(features)

features = sync_cols(X_train, features)

features = features.sort_index(axis=1)

# Make predictions on the test data
y_pred = model.predict(features)
y_prob = model.predict_proba(features)



In [24]:
# Add the labels to the predictions
prediction_output = pd.concat([labels,pd.DataFrame(y_pred,columns=["pred"]),pd.DataFrame(y_prob,columns=["proba-inv","proba"])], axis=1)

prediction_output = prediction_output.drop(columns=['proba-inv','pred'])
prediction_output = prediction_output.rename(columns={"proba":"prediction"})

prediction_output.to_csv("xgb_simple.csv", index=False)